# Cheat Sheet of Important/Common Data Modelling Steps/Metrics

# Functions

In [ ]:
# 1. Function to get a coulmn of Monday's date extracted from the week of another Date Column:
import pendulum
import datetime
from sklearn.preprocessing import LabelEncoder

def create_week(dataset, col):
    df= dataset
    col= col
    df['Week'] = df[col].isnull() # Check for missing dates

    l1=list()
    for i in range(0, len(df)):
        if (df['Week'].iloc[i] == False):
            j = df[col].iloc[i]
            today = pendulum.parse(str(j))
            start = today.start_of('week')
            l1.append(start)
        else:
            l1.append(None)
    df['Week_start']=l1
    
    df['Week_start']  = pd.to_datetime(df['Week_start'] ,utc=True, format='%Y-%m-%dT%H:%M:%S+00:00' ,errors = 'coerce')
    df['Week_start'] = df['Week_start'].dt.date
    #df['Week_start'].value_counts()
    df['Week'] = df['Week_start']
    
    # Instantiate LabelEncoder
    le = LabelEncoder()
    df['Week'] = le.fit_transform(df['Week'])

    df['Week'] = df['Week'] +1
    df['Week'].value_counts()
    

# Usage of Function:
dataset = df_dates
col = 'Date_col'
create_week(dataset, col)
df_dates[['Week', 'Week_start']].value_counts()

In [ ]:
# 2. Function to find the words with most occurrence in a String type column:
def find_common_words(dataset, col):
    df = dataset
    df[col] = df[col].astype(str)
    d = df[col].copy()

    for i in range(0, len(df)-1):
        j = df[col].iloc[i]
        d.iloc[i] = j.split() 
    #print(d)

    l=list()
    for i in range(0, len(df)-1):
        l += d.iloc[i]
    #print(l)

    #from collections import Counter
    from collections import Counter
    # Pass the split_it list to instance of Counter class. 
    Counter = Counter(l)
    # most_common() produces k frequently encountered input values and their respective counts. 
    most_occur = Counter.most_common(100) 
    print(most_occur)


# Usage of function:
dataset = df3
col = 'String_col'
find_common_words(dataset, col)

In [ ]:
# 3. Function to create categorical variables based on most frequency in categorical column(s):
def create_categories(dataset, list_col):
    list_col=list_col
    df = dataset
    for i in list_col:
        ag=df.groupby([i])['Number'].agg('count').reset_index()  
        ag=ag.sort_values(by='Number', ascending=False)
        k=list(ag.columns)
        #ag.head(4)
    
        col = 'New ' + k[0]
        df[col] = (df[k[0]].str.contains('|'.join([ag.iloc[0,0], ag.iloc[1,0], ag.iloc[2,0]]), na=False))
        df[col] = df[col].astype(object).replace({False: 0, True: 1})
        df[col] = df[col].apply(lambda x: 'Other ' + k[0] + 's' if x ==0 else x)
        df.loc[df[k[0]] ==ag.iloc[0,0],col] = ag.iloc[0,0]
        df.loc[df[k[0]] ==ag.iloc[1,0],col] = ag.iloc[1,0]
        df.loc[df[k[0]] ==ag.iloc[2,0],col] = ag.iloc[2,0]
        df[k[0]].fillna("Null", inplace = True) 
        df.loc[df[k[0]]=="Null" ,col] = "Null"

        # Check the categories to ensure nothing is left out
        #print(df[col].unique()) 
        #df[col].value_counts() 
    
        c=df[[k[0], col]]
        #print(c[c[col]==1][k[0]].unique())
        df.loc[df[col]==1 ,col] = 'Other ' + k[0] + 's'
        print(df[col].unique())
        #df[col].value_counts()

        
        
#Usage of function:
list_col = ['Col1', 'Col2', 'Col3']
dataset = df_string_cols
create_categories(dataset,list_col)

In [ ]:
# 4. Function to create a lag of one in a column(s) :
def create_lag(dataset, col_list):
    df = dataset
    for col in col_list:
        df['Lag ' + col] = 0
        for i in range(1,len(df)):
            df['Lag ' + col].iloc[i] =  df[col].iloc[i-1]
        del df[col]
        
#Usage of function:
list_col = ['Col1', 'Col2', 'Col3']
dataset = df1
create_lag(dataset,list_col)

# Algorithms Recently Used

In [ ]:
# 1.Random Forest

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20,random_state=0)

clf = RandomForestClassifier(n_estimators=10,max_depth=3, max_features='sqrt', random_state=10) # max_features: reduces the variation in accuracy for same data, random_state: reduces variation in accuracy for different train-test data
clf.fit(X_train,Y_train)
Y_pred=clf.predict(X_test)

# confusion matrix
confusion_matrix = pd.crosstab(Y_test, Y_pred, rownames=['Actual'], colnames=['Predicted'])
confusion_matrix

# Metrics of confusion matrix
from sklearn import metrics
print('Accuracy: ',metrics.accuracy_score(Y_test, Y_pred))
# Finding precision and recall
from sklearn.metrics import precision_score, recall_score, f1_score
print('Precision: %.3f' % precision_score(Y_test, Y_pred))
print('Recall: %.3f' % recall_score(Y_test, Y_pred))
print('F1 Score: %.3f' % f1_score(Y_test, Y_pred))

# First 20 important variables of Random Forest
# var ipm based on inbuilt function
sorted_idx = clf.feature_importances_.argsort()
m=pd.DataFrame(X.columns[sorted_idx], clf.feature_importances_[sorted_idx])
m.reset_index(level=0, inplace=True)
m.rename(columns={'index': 'Importance', 0: 'Variable'}, inplace=True)
cols=['Variable','Importance']
m = m.reindex(columns=cols)
m = m.sort_values(by='Importance', ascending=False)
m['Importance'] = m['Importance'].round(decimals=6)
iv=m[:20]
iv

# To get predicted probabilities
predictions = clf.predict_proba(X_test) 
predictions=pd.DataFrame(predictions)
predictions.columns=["p0","p1"]
p=predictions['p1']
p=pd.DataFrame(p,columns=['p1'])
p.columns=['p1']
Y_test=pd.DataFrame(Y_test)
df=pd.concat([Y_test.reset_index(drop=True),p.reset_index(drop=True)],axis=1)
df

In [1]:
# 2. Logistic Regression

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20,random_state=0)

lr = LogisticRegression(random_state=0).fit(X_train,Y_train)
Y_pred=lr.predict(X_test)

confusion_matrix = pd.crosstab(Y_test, Y_pred, rownames=['Actual'], colnames=['Predicted'])
confusion_matrix

from sklearn import metrics
print('Accuracy: ',metrics.accuracy_score(Y_test, Y_pred))
# Finding precision and recall
from sklearn.metrics import precision_score, recall_score, f1_score
print('Precision: %.3f' % precision_score(Y_test, Y_pred))
print('Recall: %.3f' % recall_score(Y_test, Y_pred))
print('F1 Score: %.3f' % f1_score(Y_test, Y_pred))

# Coefficients
#pd.concat([pd.DataFrame(X.columns,pd.DataFrame(np.transpose(lr.coef_)))],axis = 1)
m=pd.DataFrame(np.transpose(lr.coef_))
m['Var'] = pd.DataFrame(X.columns)
m

# Function for concordance and discordance
def concordance_discordance(model, data, predictors, outcome, testdata):       
        Probability = model.predict_proba(data[predictors])
        Probability1 = pd.DataFrame(Probability)
        Probability1.columns = ['Prob_0','Prob_1']
        TruthTable = pd.merge(data[[outcome]], Probability1[['Prob_1']], how='inner', left_index=True, right_index=True)
        zeros = TruthTable[(TruthTable[outcome]==0)].reset_index().drop(['index'], axis = 1)
        ones = TruthTable[(TruthTable[outcome]==1)].reset_index().drop(['index'], axis = 1)
        from bisect import bisect_left, bisect_right
        zeros_list = sorted([zeros.iloc[j,1] for j in zeros.index])
        zeros_length = len(zeros_list)
        disc = 0
        ties = 0
        conc = 0
        for i in ones.index:
            cur_conc = bisect_left(zeros_list, ones.iloc[i,1])
            cur_ties = bisect_right(zeros_list, ones.iloc[i,1]) - cur_conc
            conc += cur_conc
            ties += cur_ties
        pairs_tested = zeros_length * len(ones.index)
        disc = pairs_tested - conc - ties
        print('Pairs = ', pairs_tested)
        print('Conc = ', conc)
        print('Disc = ', disc)
        print('Tied = ', ties)
        concordance = round(conc/pairs_tested,2)
        discordance = round(disc/pairs_tested,2)
        ties_perc = round(ties/pairs_tested,2)
        Somers_D = round((conc - disc)/pairs_tested,2)
        print('Concordance = ', concordance*100, '%')
        print('Discordance = ', discordance*100, '%')
        print('Tied = ', ties_perc, '%')
        print('Somers D = ', Somers_D)

#Usage of above function:
x_col_list=X.columns
d = concordance_discordance(lr,df,x_col_list, 'Y_col',Y_test)
d = pd.DataFrame(d)

# AUC for Logistic Regression
from sklearn.metrics import roc_auc_score
lr_probs = df.p1
lr_auc = roc_auc_score(df['INC_SAP Occur'], lr_probs)
print('Logistic: ROC AUC=%.3f' % (lr_auc))

In [ ]:
# 3. Logistic Step Regression

from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20,random_state=0)

# Dependencies:
# In case of error while running next code snippet for newer versions of scikit:
import six
import sys
sys.modules['sklearn.externals.six'] = six
import mlrose
import joblib
sys.modules['sklearn.externals.joblib'] = joblib

#!pip install mlxtend
from mlxtend.feature_selection import SequentialFeatureSelector as sfs
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
sfs1 = sfs(lr,k_features='best',forward=True,floating=False, scoring='accuracy',cv=5) # Gives the set of features based on best value of chosen metric
sfs1 = sfs1.fit(X_train, Y_train)
feat_cols = list(sfs1.k_feature_idx_)
#print(feat_cols)#to see which columns are selected

print(feat_cols)#to see which columns are selected
f = feat_cols
l = list(sfs1.k_feature_names_)
l = pd.DataFrame(l)
l

sfs1.k_score_  # Prediction (accuracy) score of the optimal set of features

s = sfs1.subsets_ # Get all the subsets of features
type(s) # dict
feat_subset = pd.DataFrame(s)
feat_subset

# Feed all the features of best subset to logistic regressor:
X1=t[['Col1', 'Col2', 'Col5']].copy()
Y = t['Y_var'].copy()

X_train,X_test,Y_train,Y_test = train_test_split(X1,Y,test_size=0.20,random_state=0)

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state=0).fit(X_train,Y_train)
Y_pred=lr.predict(X_test)
confusion_matrix = pd.crosstab(Y_test, Y_pred, rownames=['Actual'], colnames=['Predicted'])
confusion_matrix

from sklearn import metrics
print('Accuracy: ',metrics.accuracy_score(Y_test, Y_pred))
# Finding precision and recall
from sklearn.metrics import precision_score, recall_score, f1_score
print('Precision: %.3f' % precision_score(Y_test, Y_pred))
print('Recall: %.3f' % recall_score(Y_test, Y_pred))
print('F1 Score: %.3f' % f1_score(Y_test, Y_pred))

# Coefficients
#pd.concat([pd.DataFrame(X.columns,pd.DataFrame(np.transpose(lr.coef_)))],axis = 1)
m=pd.DataFrame(np.transpose(lr.coef_))
m['Var'] = pd.DataFrame(X1.columns)
m

# For concordance/discordance, use function in Logistic Regression part.

In [ ]:
# 4. KMeans
import matplotlib.pyplot as plt
from kneed import KneeLocator  #pip install kneed
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# Scale the dataframe
features = df.copy()
scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

# Get the optimal initial number of clusters (k)
kmeans_kwargs = {
        "init": "random",
        "n_init": 10,
        "max_iter": 300,
        "random_state": 42 }
# A list holds the SSE (Sum of Squared Errors) values for each k (randomly select k=50)
sse = []
for k in range(1, 50):
    kmeans = KMeans(n_clusters=k, **kmeans_kwargs)
    kmeans.fit(scaled_features)
    sse.append(kmeans.inertia_)
# Plot the elbow graph
plt.style.use("fivethirtyeight")
plt.plot(range(1, 50), sse)
plt.xticks(range(1, 50))
plt.xlabel("Number of Clusters")
plt.ylabel("SSE")
plt.show()    
# Get the optimal value of k
kl = KneeLocator(range(1, 50), sse, curve="convex", direction="decreasing" )
kl.elbow

# Fit the kmeans model using the optimal k/selected k
kmeans = KMeans( init="random",n_clusters=5,n_init=10,max_iter=300,random_state=42)
kmeans.fit(scaled_features)

# The lowest SSE value
kmeans.inertia_

# The number of iterations required to converge
kmeans.n_iter_

# Gets cluster label of each point/row
kmeans.labels_

# Get indices of points for each cluster
{i: np.where(kmeans.labels_ == i)[0] for i in range(kmeans.n_clusters)}

# Add cluster label as a column to the data frame and convert to dummies for further analysis
t= inct.copy()
df['Y_var'] = main_df['Y_var'].copy()
df['Cluster'] = kmeans.labels_
clust = df[['Cluster']].copy()
clust = pd.get_dummies(clust['Cluster'])
col = list(clust.columns)
col = ['Cluster_' + str(sub) for sub in col]
clust.columns = col
clust

# Plot the data and cluster silhouette comparison
fig, ax1 = plt.subplots(1, figsize=(8, 6), sharex=True, sharey=True) # subplot = 1
#fig.suptitle(f"Clustering Algorithm KMeans Crescents", fontsize=16)
fte_colors = {0: "red", 1: "blue", 2: "yellow", 3: "green", 4: "black"} # number of colours must be equal to number of clusters k
# The k-means plot
km_colors = [fte_colors[label] for label in kmeans.labels_]
ax1.scatter(scaled_features[:, 0], scaled_features[:, 1], c=km_colors)
ax1.set_title(f"k-means\nSilhouette: {kmeans_silhouette}", fontdict={"fontsize": 12})

# Labeling the clusters
centers = kmeans.cluster_centers_
# Draw white circles at cluster centers
ax1.scatter(centers[:, 0], centers[:, 1], marker='o',c="white", alpha=1, s=200, edgecolor='k')
for i, c in enumerate(centers):
    ax1.scatter(c[0], c[1], marker='$%d$' % i, alpha=1,s=50, edgecolor='k')
        
plt.show()

# Plot the scatter plot of clusters and silhouette plot
cluster_labels = kmeans.fit_predict(scaled_features) # This gives a perspective into the density and separation of the formed clusters

# Silhouette score of the Kmeans model:
silhouette_avg = silhouette_score(scaled_features, cluster_labels) 
#print("For n_clusters =", n_clusters,"The average silhouette_score is :", silhouette_avg)
from sklearn.metrics import silhouette_samples
import matplotlib.cm as cm

 # Compute the silhouette scores for each sample
sample_silhouette_values = silhouette_samples(scaled_features, cluster_labels)

fig, ax1 = plt.subplots(1, figsize=(8, 6), sharex=True, sharey=True)
y_lower = 10
for i in range(kmeans.n_clusters):
    # Aggregate the silhouette scores for samples belonging to cluster i, and sort them
    ith_cluster_silhouette_values = sample_silhouette_values[cluster_labels == i]
    ith_cluster_silhouette_values.sort()

    size_cluster_i = ith_cluster_silhouette_values.shape[0]
    y_upper = y_lower + size_cluster_i

    color = cm.nipy_spectral(float(i) / kmeans.n_clusters)
    ax1.fill_betweenx(np.arange(y_lower, y_upper),0, ith_cluster_silhouette_values,facecolor=color, edgecolor=color, alpha=0.7)

    # Label the silhouette plots with their cluster numbers at the middle
    ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i))

    # Compute the new y_lower for next plot
    y_lower = y_upper + 10  # 10 for the 0 samples

ax1.set_title("The silhouette plot for the various clusters.")
ax1.set_xlabel("The silhouette coefficient values")
ax1.set_ylabel("Cluster label")

# The vertical line for average silhouette score of all the values
ax1.axvline(x=silhouette_avg, color="red", linestyle="--")

ax1.set_yticks([])  # Clear the yaxis labels / ticks
ax1.set_xticks([-0.1, 0, 0.2, 0.4, 0.6, 0.8, 1])

# Get silhouette values for all records
f = pd.DataFrame(list(sample_silhouette_values))
f['Cluster'] = list(kmeans.labels_)
f

# Compute the silhouette scores for each algorithm
from sklearn.metrics import silhouette_score
kmeans_silhouette = silhouette_score(scaled_features, kmeans.labels_).round(2)
kmeans_silhouette

In [ ]:
# 5. Decision Tree
# Split dataset into training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
# Create Decision Tree classifer object
clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train,y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))

# Representation of Decisison Tree in the form of text
text_representation = tree.export_text(clf)
print(text_representation)

# Representation of Decision Tree in the form of image
#conda install python-graphviz
#pip install six

from sklearn.tree import export_graphviz
#from sklearn.externals.six import StringIO   ### Has dependency issues based on version of package ###
from six import StringIO
from IPython.display import Image  
import pydotplus

dot_data = StringIO()
export_graphviz(clf, out_file="tree.dot",  
                filled=True, rounded=True,
                special_characters=True,feature_names = X.columns,class_names=['0','1'])
 
#graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
#graph.write_png('SAP.png')
#Image(graph.create_png())

!dot -Tpng tree.dot -o tree.png -Gdpi=600

Image(filename = 'tree.png') # Image stored in the file being used

In [ ]:
# 6. Time Series - Unobserved Components
df = features.copy()
train = df.drop(['Y_var'],axis=1)
test=pd.DataFrame(df['Y_var'].copy())

train1=train[0:90]
train2 = train[90:]

test1=test[:90]
test2=test[90:]

import statsmodels.api as sm
from statsmodels.tsa.statespace.structural import UnobservedComponents

#multi variate
unrestricted_model = {'level':'random trend', 'cycle': False, 'damped_cycle': False, 'stochastic_cycle': True,'autoregressive':2}

model = sm.tsa.UnobservedComponents(endog=test1, exog=train1, **unrestricted_model)
model_fit=model.fit(method='bfgs',disp=False)

pred = model_fit.predict(dynamic=True,exog=train2, start = len(train1), end =len(train1) + len(train2)-1)
pred

# Mean Squared Error
from math import sqrt
from sklearn.metrics import mean_squared_error 
mse=sqrt(mean_squared_error(pred, test2))
print('Mean Squared Error: ' + str(mse))

# get correlation between all independent vars and dependent var
X = df.drop(['Y_var'], axis=1)
Y = df['Y_var'].copy()
cor=X.corrwith(Y)
cor

# Miscellaneous

In [ ]:
# Create a column based on conditions in another String type column:
df1['Col3'] = (df1['Col1'].str.contains('Hello', na=False)) & (df1['Col2'].str.contains('Hi', na=False))
df1['Col3'] = df1['Col3'].astype(object).replace({False: 0, True: 1}) 
df1['Col3'].value_counts()

# Replace a value in an entire column:
df3['Col1'] = df3['Col1'].replace(['Hi'],'Hello')

# Replace value of a String type column based on a condition on another String type column:
df.loc[df['Col1'].str.contains('Hi', na=False), 'Col2'] = 'Hello'

In [ ]:
# To convert a column to datetime
df1['Date2']=pd.to_datetime(df2['Date1'], format='%d-%m-%Y')    # 'Format' is the format of the dates present in the columns initially.
                                                                # It has to be mentioned, otherwise pd.to_datetime will convert into random formats

In [ ]:
# Get the hour from datetime column:
df3['Hour']=df3.Date_column.dt.hour

# Create column having part of the day based on hour:
df3['Time of Day'] = 0
df3.loc[(df3.Hour >= 6) & (df3.Hour < 12), 'Time of Day'] = 'Morning'
df3.loc[(df3.Hour >= 12) & (df3.Hour < 18), 'Time of Day'] = 'Afternoon'
df3.loc[(df3.Hour >= 18) & (df3.Hour < 24), 'Time of Day'] = 'Evening'
df3.loc[df3.Hour < 6 , 'Time of Day'] = 'Night'

In [ ]:
# Get number of business days between two date columns (excluding weekends):
from pandas.tseries.offsets import *
#pd.date_range('2011-01-05', '2011-01-09', freq=BDay()).shape[0]  ----to get diff in days without weekends for 2 dates
df3['No. of Business Days'] = 0
### USE LOC TO FIND NON-NULL ROWS
df3.loc[(df3.DateA.notnull()) & (df3.DateB.notnull()), 'No. of Business Days'] = df3[(df3.DateA.notnull()) & (df3.DateB.notnull())].apply(lambda row: len(pd.bdate_range(row['DateA'], row['DateB'])), axis = 1)

In [ ]:
# Merge tables:
df3=pd.merge(df1,df2,how='left', on= 'Common_column')   # how = inner/outer/left/right

# Concat tables along columns:
df = pd.concat([df1,df2,df3], axis=1)   # axis=0 to concat along rows

In [ ]:
# To find correlation between input variable columns and output variable column:
# Inbuilt pandas function corrwith to get all correlation between independent vars and dependent var
X = df.drop(['Y_var'], axis=1)
Y = df['Y_var'].copy()
cor=X.corrwith(Y)

In [ ]:
# Diff of no. of events between present & the last row:
df['Increase_in_count'] = df['No. of events'].diff(periods = 1)  # If periods=2, then finds difference between last and last-2 events

In [ ]:
# Convert Categorical columns to Dummy variables using the categories in it:
x = df[['col1','col2','col3','col4']].copy()
x1=pd.get_dummies(x)

In [ ]:
# Add a substring preceding all column names of a dataframe:
col = list(df.columns)
col = ['A-Z' + sub for sub in col]
df.columns = col

# Reset index of dataframe:
df.reset_index(level=1, inplace=True)
df.columns